In [7]:
# https://www.edureka.co/blog/web-scraping-with-python/
# https://stackoverflow.com/questions/34519746/using-python-docx-to-update-cell-content-of-a-table
# https://www.lambdatest.com/blog/edge-driver/

# TODO: check if edgedriver version is up-to-date with edge version--> https://developer.microsoft.com/de-de/microsoft-edge/tools/webdriver/

from selenium import webdriver
from bs4 import BeautifulSoup
from docx import Document, enum
from docx.shared import Pt

doc_name = "Vorlage_S.25.docx"
rows_available = 14 # both sections need to have the same number of rows available
start_row_club_goalgetter_1 = 1
start_row_club_goalgetter_2 = rows_available + start_row_club_goalgetter_1 + 1

# URLS needs to be updated every season
club_goalgetter = [
{"url": "https://www.fupa.net/team/sg-mallersdorf-grafentr-m1-2023-24/scorers", "start_row": start_row_club_goalgetter_1, "start_column": 0 },              
{"url": "https://www.fupa.net/team/tsv-pilsting-m1-2023-24/scorers",  "start_row": start_row_club_goalgetter_2, "start_column": 0 },              
{"url": "https://www.fupa.net/team/sv-wendelskirchen-m1-2023-24/scorers", "start_row": start_row_club_goalgetter_1, "start_column": 2 },
{"url": "https://www.fupa.net/team/fc-wallersdorf-m1-2023-24/scorers", "start_row": start_row_club_goalgetter_2, "start_column": 2 },             
{"url": "https://www.fupa.net/team/ssv-1983-weng-m1-2023-24/scorers", "start_row": start_row_club_goalgetter_1, "start_column": 4 },
{"url": "https://www.fupa.net/team/sv-wallkofen-m1-2023-24/scorers", "start_row": start_row_club_goalgetter_2, "start_column": 4 },
{"url": "https://www.fupa.net/team/spvgg-haberskirchen-m1-2023-24/scorers", "start_row": start_row_club_goalgetter_1, "start_column": 6 },        
{"url": "https://www.fupa.net/team/fc-ottering-m1-2023-24/scorers", "start_row": start_row_club_goalgetter_2, "start_column": 6 },       
{"url": "https://www.fupa.net/team/sv-grosskoellnbach-m1-2023-24/scorers", "start_row": start_row_club_goalgetter_1, "start_column": 8 },   
{"url": "https://www.fupa.net/team/tsv-mamming-m1-2023-24/scorers", "start_row": start_row_club_goalgetter_2, "start_column": 8 },  
{"url": "https://www.fupa.net/team/tsv-hofkirchen-m1-2023-24/scorers", "start_row": start_row_club_goalgetter_1, "start_column": 10 }, 
{"url": "https://www.fupa.net/team/sg-pfaffenberg-oberlindhart-m1-2023-24/scorers", "start_row": start_row_club_goalgetter_2, "start_column": 10 }, 
{"url": "https://www.fupa.net/team/fc-gottfrieding-m1-2023-24/scorers", "start_row": start_row_club_goalgetter_1, "start_column": 12 },
{"url": "https://www.fupa.net/team/sg-griesbach-steinberg-m1-2023-24/scorers", "start_row": start_row_club_goalgetter_2, "start_column": 12 } ]

goalgetter_url = "https://www.fupa.net/league/kreisklasse-dingolfing/scorers"

def print_data(position, name, club, goals, assists, games):
    print('position:')
    print(position)
    print('name')
    print(name)
    print('club')
    print(club)
    print('goals')
    print(goals)
    print('assists')
    print(assists)
    print('games')
    print(games)

# fix name mapping - data based on goalgetter_url
def fix_club_name(name):
    if name == "SG Pfaffenberg":
        name = "SG Pfaffenberg-Oberlindhart"
    if name == "Großköllnbac":
        name = "SV Großköllnbach"
    if name == "Hofkirchen":
        name = "TSV Hofkirchen"
    if name == "SG Mallersdorf":
        name = "SG Mallersdorf-Grafentr."
    if name == "Wallersdorf":
        name = "FC Wallersdorf"
    if name == "Weng":
        name = "SSV Weng"
    if name == "Haberskirch.":
        name = "SpVgg Haberskirchen"
    if name == "Mamming":
        name = "TSV Mamming"
    if name == "Wallkofen":
        name = "SV Wallkofen"
    if name == "Ottering":
        name = "FC Ottering"
    if name == "Gottfrieding":
        name = "FC Gottfrieding"
    if name == "SG Griesbach":
        name = "SG Griesbach-Steinberg"
    if name == "Pilsting":
        name = "TSV Pilsting"
    if name == "Wendelskir.":
        name = "SV Wendelskirchen"
    return name

def set_font(doc, font_name):
    style = doc.styles['Normal']
    font = style.font
    font.name = font_name

def update_goalgetter(doc):

    # init steps
    driver = webdriver.Edge()
    driver.get(goalgetter_url)
    content = driver.page_source
    soup = BeautifulSoup(content)

    # find all goalgetter rows
    rows = soup.find_all("div", {"class": "sc-1mrugnb-15 hczLZA"}) 
    if rows is None or len(rows) == 0:
        raise Exception('No goalgetter rows found')
    
    # print('Goalgetter rows:')
    # print(rows)

    # init variables for goalgetter infos
    position, name, club, goals, assists, games = [], [], [], [], [], []

    # fill variables with goalgetter infos
    for r in rows:
        # position
        position_element = r.find("div", {"class": "sc-1mrugnb-12 jrFtoU"})
        if position_element is None:
            raise Exception('No position_element found')
        position.append(position_element.text)

        # name
        name_element = r.find("span", {"class": "sc-lhxcmh-0 jOiTFY sc-1mrugnb-16 sPeDb"})
        if name_element is None:
            raise Exception('No name_element found')
        name.append(name_element.text)

        # club
        club_element = r.find("span", {"class": "sc-lhxcmh-0 hudpXq sc-1mrugnb-16 sPeDb"})
        if club_element is None:
            raise Exception('No club_element found')
        club.append(fix_club_name(club_element.text))

        # goals, assists and games
        statistics_element = r.find_all("div", {"class": "sc-1mrugnb-12 fvlIFr"})
        if statistics_element is None or len(statistics_element) == 0:
            raise Exception('No statistics_element found')
        if len(statistics_element) != 3:
            raise Exception('Not enough statistics_element found')
        goals.append(statistics_element[0].text)
        assists.append(statistics_element[1].text)
        games.append(statistics_element[2].text)

    print('Getting goalgetter infos successful')
    print_data(position, name, club, goals, assists, games)

    table = doc.tables[1]

    for x in range(19):
        # skip empty rows
        if x % 2 == 0:
            for y in range(7):
                if y == 1 :
                    table.cell(x,y).text = name[int(x/2)]
                elif y == 2:
                    table.cell(x,y).text = club[int(x/2)]
                elif y == 3:
                    table.cell(x,y).text = goals[int(x/2)]
                elif y == 5:
                    table.cell(x,y).text = assists[int(x/2)]
                table.cell(x,y).paragraphs[0].runs[0].font.size = Pt(8)
                table.cell(x,y).paragraphs[0].runs[0].font.bold = True

def update_club_goalgetter(doc):
    driver = webdriver.Edge()
    table = doc.tables[0]

    for i in range(len(club_goalgetter)):

        # init steps
        driver.get(club_goalgetter[i]["url"])
        content = driver.page_source
        soup = BeautifulSoup(content)

        # find the club goalgetter rows
        rows = soup.find_all("div", {"class": "sc-1mrugnb-15 hczLZA"})
        if rows is None or len(rows) == 0:
            raise Exception('No club goalgetter rows found for ' + club_goalgetter[i]["url"])

        # init variables for club goalgetter infos
        name, goals = [], []

        # fill variables with club goalgetter infos
        for r in rows:
            # name of club goalgetter
            name_element = r.find("span", {"class": "sc-lhxcmh-0 jOiTFY sc-1mrugnb-16 sPeDb"})
            if name_element is None:
                raise Exception('No name_element found')
            name.append(name_element.text)

            # goals of club goalgetter
            statistics_element = r.find_all("div", {"class": "sc-1mrugnb-12 fvlIFr"})
            if statistics_element is None or len(statistics_element) == 0:
                raise Exception('No statistics_element found')
            goals.append(statistics_element[0].text)

        print('Getting club goalgetter infos successful for ' + club_goalgetter[i]["url"])

        if len(name) > rows_available:
            raise Exception("Not enough rows available, you need to add more rows!" + "length of name array: " + len(name) + " and rows_available: " + rows_available)           
        
        for j in range(len(name)):
            cell_name = table.cell(club_goalgetter[i]["start_row"] + j, club_goalgetter[i]["start_column"])
            cell_name.text = name[j]
            cell_name.paragraphs[0].runs[0].font.size = Pt(6)
            cell_goals = table.cell(club_goalgetter[i]["start_row"] + j, club_goalgetter[i]["start_column"] + 1)
            cell_goals.text = goals[j]
            cell_goals.paragraphs[0].runs[0].font.size = Pt(6)
            cell_goals.paragraphs[0].alignment = enum.text.WD_ALIGN_PARAGRAPH.CENTER
            print("Name: ", name[j], " with goals: ", goals[j])

def init():
    doc = Document(doc_name)
    set_font(doc, 'Arial')
    update_club_goalgetter(doc)
    update_goalgetter(doc)
    doc.save("_S.25.docx")

init()

Getting club goalgetter infos successful for https://www.fupa.net/team/sg-mallersdorf-grafentr-m1-2023-24/scorers
Name:  M. Löffler  with goals:  8
Name:  F. Prebeck-Sanchez  with goals:  8
Name:  E. Daffner  with goals:  4
Name:  K. Prebeck-Sanchez  with goals:  4
Name:  P. Kleinevoss  with goals:  3
Name:  L. Löffler  with goals:  2
Name:  C. Grauschopf  with goals:  2
Name:  L. Gruber  with goals:  1
Name:  B. Schütz  with goals:  1
Getting club goalgetter infos successful for https://www.fupa.net/team/tsv-pilsting-m1-2023-24/scorers
Name:  K. Gegenfurtner  with goals:  5
Name:  T. Hofmann  with goals:  4
Name:  J. Aulinger  with goals:  2
Name:  F. Bendzko  with goals:  1
Name:  M. Käser  with goals:  1
Name:  M. Gabler  with goals:  1
Name:  R. Ritzinger  with goals:  1
Getting club goalgetter infos successful for https://www.fupa.net/team/sv-wendelskirchen-m1-2023-24/scorers
Name:  D. Laubner  with goals:  3
Name:  D. Hitzenberger  with goals:  2
Name:  M. Samberger  with goals: 